In [20]:
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import pandas as pd
import seaborn as sns
import matplotlib
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import learning_curve
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [70]:
data = pd.read_csv('dataframe2.csv')

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
data = df.sort_values(by=['date'])
data

,date,file_id,target,ndvi_mean,ndvi_median,ndvi_std,mi_mean,mi_median,mi_std,bc1_mean,...,b10_mean,b10_median,b10_std,b11_mean,b11_median,b11_std,b12_mean,b12_median,b12_std,month
1786,2020-07-03,Name_5f2a5324868954001c94d201,sugarcane,0.250802,0.253114,0.014855,0.264690,0.266351,0.011328,5063.663810,...,3772.030476,0.0,4485.894054,1154.457143,0.0,1374.713478,889.934286,0.0,1059.201426,7
2405,2020-07-03,id_6054a1b558a7cd00480009b9,sugarcane,0.045984,0.045692,0.003819,0.283422,0.283140,0.003703,15624.181818,...,1220.886364,1465.0,553.602447,2960.727273,3568.0,1342.194876,1716.125000,2068.0,777.945543,7
1801,2020-07-03,id_6054a1b458a7cd00480009a1,sugarcane,0.051992,0.051309,0.005102,0.253422,0.253802,0.004671,13752.631687,...,1098.549383,1389.0,570.094574,2770.154321,3507.0,1436.938595,1679.954733,2120.0,871.491294,7
3206,2020-07-03,id_6054a1b458a7cd004800099b,sugarcane,0.066951,0.065891,0.007054,0.271170,0.270490,0.004760,13180.242424,...,1030.560606,1233.0,473.999780,2625.348485,3186.0,1207.604328,1564.287879,1904.0,719.816281,7
2429,2020-07-03,id_6054a1b558a7cd00480009d7,sugarcane,0.076495,0.033484,0.462855,0.304458,0.304456,0.005560,12461.900000,...,3700.795238,1584.0,5064.699268,2230.102857,3527.0,1799.318805,1361.360000,2036.0,1110.769679,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1671,2021-06-30,id_6054a1b458a7cd004800099b,sugarcane,0.440827,0.461003,0.092351,0.126450,0.128659,0.044161,7713.303030,...,4947.174242,5949.0,2275.804771,2618.681818,3027.5,1227.776472,1947.916667,2265.0,934.167617,6
2485,2021-06-30,id_6054a1b558a7cd00480009d7,sugarcane,0.438477,0.449263,0.058814,3.513647,0.063961,4.848746,4479.320952,...,2181.831429,1514.0,2234.713163,1897.510476,2943.0,1535.442660,1295.976190,1925.0,1059.287537,6
2502,2021-06-30,id_6054a1b558a7cd00480009b9,sugarcane,0.513206,0.537832,0.083370,0.146012,0.155046,0.033941,6588.988636,...,4498.829545,5400.0,2040.409435,2211.022727,2648.5,1004.824589,1487.636364,1738.5,683.302828,6
2419,2021-06-30,id_6054a1b558a7cd00480009e9,sugarcane,0.399044,0.377174,0.086154,5.960522,0.183560,6.521667,4841.391667,...,1107.436905,1425.0,607.022327,1982.441667,2534.0,1088.129617,1396.050000,1770.0,768.990515,6


In [3]:
df = pd.read_csv('dataframe3.csv')
df

,date,id,target,ndvi_mean,ndvi_std,ndvi_median,mi_mean,mi_std,mi_median,bc1_mean,...,b10_mean,b10_std,b10_median,b11_mean,b11_std,b11_median,b12_mean,b12_std,b12_median,month
0,2021-01-29,id_6054a1b558a7cd00480009c5,sugarcane,0.326887,0.039961,0.333059,0.356040,0.056311,0.370403,6652.833333,...,0.880952,0.323845,1.0,0.880952,0.323845,1.0,0.880952,0.323845,1.0,1
1,2021-01-01,id_6054a1b558a7cd00480009cb,sugarcane,0.389033,0.085019,0.370453,0.388601,0.084351,0.389844,5773.906250,...,0.742188,0.437430,1.0,0.742188,0.437430,1.0,0.742188,0.437430,1.0,1
2,2021-01-31,id_6054a1b558a7cd00480009cb,sugarcane,0.231330,0.081022,0.187807,4.498033,5.008589,0.353001,5645.617188,...,0.742188,0.437430,1.0,0.742188,0.437430,1.0,0.742188,0.437430,1.0,1
3,2021-01-16,id_6054a1b558a7cd00480009b0,sugarcane,0.802878,5.477423,0.047894,24.260523,8.707744,26.798188,2048.275815,...,0.614130,0.486800,1.0,0.614130,0.486800,1.0,0.614130,0.486800,1.0,1
4,2021-01-19,id_6054a1b558a7cd00480009d7,sugarcane,0.314713,0.077776,0.302136,0.322170,0.149583,0.303254,4964.931429,...,0.606667,0.488490,1.0,0.606667,0.488490,1.0,0.606667,0.488490,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7788,2020-12-26,id_60a09545f85c5e001b8714ba,rice_1y,6.889612,4.426219,9.179983,0.097189,0.009057,0.097371,2701.352881,...,0.272634,0.445314,0.0,0.272634,0.445314,0.0,0.272634,0.445314,0.0,12
7789,2020-12-05,id_6054a1b558a7cd00480009c8,sugarcane,0.447698,0.063686,0.453211,0.431491,0.095524,0.415568,5331.385185,...,0.729630,0.444151,1.0,0.729630,0.444151,1.0,0.729630,0.444151,1.0,12
7790,2020-12-31,id_6094f8d3f85c5e001b870a62,rice_1y,0.302445,0.048487,0.314660,0.415134,0.035912,0.417534,724.588387,...,0.136774,0.343609,0.0,0.136774,0.343609,0.0,0.136774,0.343609,0.0,12
7791,2020-12-01,id_6094c0dcf85c5e001b870958,rice_1y,17.307417,4.500575,18.294807,0.336959,0.013345,0.334452,3713.126935,...,0.684211,0.464830,1.0,0.684211,0.464830,1.0,0.684211,0.464830,1.0,12


In [4]:
df['month'] = pd.DatetimeIndex(df['date']).month
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df = df.sort_values(by=['date'])

In [5]:
df

,date,id,target,ndvi_mean,ndvi_std,ndvi_median,mi_mean,mi_std,mi_median,bc1_mean,...,b10_mean,b10_std,b10_median,b11_mean,b11_std,b11_median,b12_mean,b12_std,b12_median,month
3863,2020-07-03,id_6054a1b558a7cd00480009ad,sugarcane,0.089297,0.008006,0.087037,0.527489,0.006609,0.527061,10514.746212,...,0.598485,0.490205,1.0,0.598485,0.490205,1.0,0.598485,0.490205,1.0,7
4059,2020-07-03,id_6054a1b558a7cd00480009ce,sugarcane,0.072480,0.005242,0.071939,0.552065,0.002738,0.551209,13295.838235,...,0.683824,0.464983,1.0,0.683824,0.464983,1.0,0.683824,0.464983,1.0,7
4061,2020-07-03,Name_5f2a4ef2868954001c94d1d5,sugarcane,0.094622,0.006876,0.093762,0.522520,0.004568,0.522812,12855.553571,...,0.794643,0.403962,1.0,0.794643,0.403962,1.0,0.794643,0.403962,1.0,7
4108,2020-07-03,id_6054a1b458a7cd00480009a4,sugarcane,0.098188,0.005333,0.098000,0.507541,0.002160,0.507661,10967.027778,...,0.627778,0.483397,1.0,0.627778,0.483397,1.0,0.627778,0.483397,1.0,7
4112,2020-07-03,id_6054a1b558a7cd00480009bf,sugarcane,0.077317,0.004381,0.077215,0.574045,0.002810,0.574301,15025.526611,...,0.775910,0.416981,1.0,0.775910,0.416981,1.0,0.775910,0.416981,1.0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3553,2021-06-30,id_6054a1b558a7cd00480009b6,sugarcane,0.364726,0.050160,0.372745,0.351492,0.046100,0.356412,4430.000000,...,0.657051,0.474695,1.0,0.657051,0.474695,1.0,0.657051,0.474695,1.0,6
3560,2021-06-30,id_6054a1b558a7cd00480009b3,sugarcane,0.335830,0.040321,0.342617,0.298701,0.039708,0.297269,4645.349490,...,0.557398,0.496695,1.0,0.557398,0.496695,1.0,0.557398,0.496695,1.0,6
3792,2021-06-30,id_6054a1b558a7cd00480009ad,sugarcane,0.315349,0.037817,0.313292,0.292189,0.040127,0.286434,5797.649621,...,0.598485,0.490205,1.0,0.598485,0.490205,1.0,0.598485,0.490205,1.0,6
3515,2021-06-30,id_6054a1b558a7cd00480009c5,sugarcane,0.359492,0.052419,0.354348,0.301320,0.062846,0.293834,7710.452381,...,0.880952,0.323845,1.0,0.880952,0.323845,1.0,0.880952,0.323845,1.0,6


In [6]:
df.keys()

Index(['date', 'id', 'target', 'ndvi_mean', 'ndvi_std', 'ndvi_median',
       'mi_mean', 'mi_std', 'mi_median', 'bc1_mean', 'bc1_std', 'bc1_median',
       'bc2_mean', 'bc2_std', 'bc2_median', 'bc3_mean', 'bc3_std',
       'bc3_median', 'b0_mean', 'b0_std', 'b0_median', 'b1_mean', 'b1_std',
       'b1_median', 'b2_mean', 'b2_std', 'b2_median', 'b3_mean', 'b3_std',
       'b3_median', 'b4_mean', 'b4_std', 'b4_median', 'b5_mean', 'b5_std',
       'b5_median', 'b6_mean', 'b6_std', 'b6_median', 'b7_mean', 'b7_std',
       'b7_median', 'b8_mean', 'b8_std', 'b8_median', 'b9_mean', 'b9_std',
       'b9_median', 'b10_mean', 'b10_std', 'b10_median', 'b11_mean', 'b11_std',
       'b11_median', 'b12_mean', 'b12_std', 'b12_median', 'month'],
      dtype='object')

In [11]:
columns = ['date', 'file_id', 'target', 'ndvi_mean', 'ndvi_std', 'ndvi_median',
       'mi_mean', 'mi_std', 'mi_median', 'bc1_mean', 'bc1_std', 'bc1_median',
       'bc2_mean', 'bc2_std', 'bc2_median', 'bc3_mean', 'bc3_std',
       'bc3_median','b1_mean', 'b1_std',
       'b1_median', 'b2_mean', 'b2_std', 'b2_median', 'b3_mean', 'b3_std',
       'b3_median', 'b4_mean', 'b4_std', 'b4_median', 'b5_mean', 'b5_std',
       'b5_median', 'b6_mean', 'b6_std', 'b6_median', 'b7_mean', 'b7_std',
       'b7_median', 'b8_mean', 'b8_std', 'b8_median', 'b9_mean', 'b9_std',
       'b9_median', 'b10_mean', 'b10_std', 'b10_median', 'b11_mean', 'b11_std',
       'b11_median', 'b12_mean', 'b12_std', 'b12_median','b13_mean', 'b13_std', 'b13_median','month']

In [12]:
df2 = df.copy()
df2.columns = columns
df2

,date,file_id,target,ndvi_mean,ndvi_std,ndvi_median,mi_mean,mi_std,mi_median,bc1_mean,...,b10_mean,b10_std,b10_median,b11_mean,b11_std,b11_median,b12_mean,b12_std,b12_median,month
3863,2020-07-03,id_6054a1b558a7cd00480009ad,sugarcane,0.089297,0.008006,0.087037,0.527489,0.006609,0.527061,10514.746212,...,0.598485,0.490205,1.0,0.598485,0.490205,1.0,0.598485,0.490205,1.0,7
4059,2020-07-03,id_6054a1b558a7cd00480009ce,sugarcane,0.072480,0.005242,0.071939,0.552065,0.002738,0.551209,13295.838235,...,0.683824,0.464983,1.0,0.683824,0.464983,1.0,0.683824,0.464983,1.0,7
4061,2020-07-03,Name_5f2a4ef2868954001c94d1d5,sugarcane,0.094622,0.006876,0.093762,0.522520,0.004568,0.522812,12855.553571,...,0.794643,0.403962,1.0,0.794643,0.403962,1.0,0.794643,0.403962,1.0,7
4108,2020-07-03,id_6054a1b458a7cd00480009a4,sugarcane,0.098188,0.005333,0.098000,0.507541,0.002160,0.507661,10967.027778,...,0.627778,0.483397,1.0,0.627778,0.483397,1.0,0.627778,0.483397,1.0,7
4112,2020-07-03,id_6054a1b558a7cd00480009bf,sugarcane,0.077317,0.004381,0.077215,0.574045,0.002810,0.574301,15025.526611,...,0.775910,0.416981,1.0,0.775910,0.416981,1.0,0.775910,0.416981,1.0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3553,2021-06-30,id_6054a1b558a7cd00480009b6,sugarcane,0.364726,0.050160,0.372745,0.351492,0.046100,0.356412,4430.000000,...,0.657051,0.474695,1.0,0.657051,0.474695,1.0,0.657051,0.474695,1.0,6
3560,2021-06-30,id_6054a1b558a7cd00480009b3,sugarcane,0.335830,0.040321,0.342617,0.298701,0.039708,0.297269,4645.349490,...,0.557398,0.496695,1.0,0.557398,0.496695,1.0,0.557398,0.496695,1.0,6
3792,2021-06-30,id_6054a1b558a7cd00480009ad,sugarcane,0.315349,0.037817,0.313292,0.292189,0.040127,0.286434,5797.649621,...,0.598485,0.490205,1.0,0.598485,0.490205,1.0,0.598485,0.490205,1.0,6
3515,2021-06-30,id_6054a1b558a7cd00480009c5,sugarcane,0.359492,0.052419,0.354348,0.301320,0.062846,0.293834,7710.452381,...,0.880952,0.323845,1.0,0.880952,0.323845,1.0,0.880952,0.323845,1.0,6


In [13]:
#GroupShuffleSplit
from sklearn.model_selection import GroupShuffleSplit
train_inds, test_inds = next(GroupShuffleSplit(test_size=.20, n_splits=2, random_state = 7).split(df2, groups=df2['file_id']))
train = df2.iloc[train_inds]
test = df2.iloc[test_inds]

In [15]:
train.shape

(6045, 58)

In [16]:
test.shape

(1748, 58)

In [22]:
X_train = train.drop(columns = ['target','file_id', 'date'])
y_train= train['target']

X_test = test.drop(columns = ['target','file_id','date'])
y_test = test['target']

In [23]:
A = train.file_id.unique()
B = test.file_id.unique()
bool(set(A).intersection(B))

False

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

pipe = Pipeline([('scaler', RobustScaler()),('pca', PCA()),('knn', KNeighborsClassifier(n_neighbors=4, leaf_size = 1, p = 1))])
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

pipe.score(X_test,y_test)

# cm = confusion_matrix(y_test, y_pred)
# print(cm)
print('Accuracy',accuracy_score(y_test, y_pred))

Accuracy 0.9130434782608695


In [25]:
# Save model 
import joblib
# save the model to disk
filename ='pipeline_test.joblib'
joblib.dump(pipe,filename)
print("saved pipeline.joblib locally")

saved pipeline.joblib locally


In [28]:
# load the model from disk
loaded_model = joblib.load(filename)
result = loaded_model.score(X_test,y_test)
print(result)

0.9130434782608695
